In [1]:
!pip install XlsxWriter

import csv
import re
import xlsxwriter

     |████████████████████████████████| 143kB 2.8MB/s 


In [2]:
fn = "POS-Report-Detail Outlet-201701"

data = []
with open(f"{fn}.csv") as csvfile:
  readcsv = csv.reader(csvfile)
  for row in readcsv:
    data.append(row)

In [3]:
code_pattern = re.compile("(RM|FD|BV|OT)\d{3,4}")  # RM, FD, BV, OT

workbook = xlsxwriter.Workbook(f'{fn}-clean.xlsx')
worksheet = workbook.add_worksheet(fn)

# write the column names
col_names = ["Code", "Description", "Qty", "Food Rev", "Bev Rev", "Oth FB Rev", 
             "Amount +/+", "SvChg", "Tax", "Amount Nett", "Cash", "Credit Card",
             "City Ledger", "Bill FO", "Other Pymt", "Remarks", "Bill No"]
for col, name in enumerate(col_names):
  worksheet.write(0, col, name)

# iterate data
row = 1
for dat in data:

  # check whether contain Bill No information
  # this will start record bill details
  if dat[0] == 'Bill No :':
    bill = dat[2]
    bill_details = []
    bill_end = False
    continue
  
  # check whether contains true code pattern
  # if so, append into bill details
  elif code_pattern.fullmatch(dat[0]) != None:
    bill_details.append(dat)
    continue

  # check whether it is the end of bill details
  elif (dat[0] == '') and (dat[3] == 'Total') and (len(bill_details)>0) and \
      not bill_end:
    
    # clean numeric information string into integer format
    try: cash = int(float(dat[21].replace(" ", "")))
    except: cash = 0
    try: credit_card = int(float(dat[23].replace(" ", "")))
    except: credit_card = 0
    try: city_ledger = int(float(dat[25].replace(" ", "")))
    except: city_ledger = 0
    try: bill_fo = int(float(dat[27].replace(" ", "")))
    except: bill_fo = 0
    try: other_pymt = int(float(dat[29].replace(" ", "")))
    except: other_pymt = 0
    remarks = dat[31]
    if remarks == "": remarks = "Not Remarks"

    bill_end = True

    # write all details into xlsx
    for bd in bill_details:
      for i, x in enumerate([bd[0], bd[2],
                             int(float(bd[8].replace(" ", ""))), 
                             int(float(bd[9].replace(" ", ""))),
                             int(float(bd[11].replace(" ", ""))), 
                             int(float(bd[12].replace(" ", ""))),
                             int(float(bd[14].replace(" ", ""))), 
                             int(float(bd[15].replace(" ", ""))), 
                             int(float(bd[17].replace(" ", ""))), 
                             int(float(bd[19].replace(" ", ""))),
                             cash, credit_card, city_ledger, bill_fo,
                             other_pymt, remarks, bill]):
        worksheet.write(row, i, x)
      row += 1

workbook.close()

In [4]:
import xlrd

workbook = xlrd.open_workbook("POS-Report-Detail Outlet-201701-clean.xlsx")
sheet = workbook.sheet_by_name("POS-Report-Detail Outlet-201701")

for rowx in range(sheet.nrows):
    values = sheet.row_values(rowx)
    print(values)

['Code', 'Description', 'Qty', 'Food Rev', 'Bev Rev', 'Oth FB Rev', 'Amount +/+', 'SvChg', 'Tax', 'Amount Nett', 'Cash', 'Credit Card', 'City Ledger', 'Bill FO', 'Other Pymt', 'Remarks', 'Bill No']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 714876.0, 714876.0, 71488.0, 78636.0, 865000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'HA101', 'PC-00000172']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 371901.0, 371901.0, 37190.0, 40909.0, 450000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'K103', 'PC-00000173']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 504132.0, 504132.0, 50413.0, 55455.0, 610000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'K104', 'PC-00000174']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 714876.0, 714876.0, 71488.0, 78636.0, 865000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'K105', 'PC-00000175']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 396694.0, 396694.0, 39669.0, 43636.0, 480000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'K106', 'PC-00000176']
['RM001', 'Room Revenue', 1.0, 0.0, 0.0, 396694.0, 396694.0, 39669.0, 43636.0, 480000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'K

In [12]:
temp = [sheet.row_values(rowx)[-1] for rowx in range(1, sheet.nrows)]

In [13]:
max([len(x) for x in temp])

12